## SRJ - Building H using HF integrals

In [33]:
import basis_set_exchange as bse

basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')

import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

# Your original setup (unchanged)
mol_pyscf = gto.M(
    atom = 'Fe 0 0 0; S 0 0 1.826',
    basis = basis_dict,
    spin = 4)
rhf = scf.UHF(mol_pyscf)
rhf.kernel()

converged SCF energy = -1655.10155370991  <S^2> = 6.0386361  2S+1 = 5.0154306


np.float64(-1655.1015537099054)

In [23]:
one_ao = mol_pyscf.intor_symmetric('int1e_kin') + mol_pyscf.intor_symmetric('int1e_nuc') #Kinetic energy integrals + Nuclear energy integrals = Hcore
two_ao = mol_pyscf.intor('int2e_sph') # Two electron integrals

print(rhf.mo_coeff.shape)  # Should be (num_AOs, num_MOs)
print(one_ao.shape)  
print(two_ao.shape)

one_mo = np.einsum('pi,pq,qj->ij', rhf.mo_coeff, one_ao, rhf.mo_coeff)
#one_mo = np.einsum('spi,pq,sqj->sij', rhf.mo_coeff, one_ao, rhf.mo_coeff) #Changed this 
#one_mo = np.einsum('...pi,pq,...qj->...ij', rhf.mo_coeff, one_ao, rhf.mo_coeff)

two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

two_mo = np.swapaxes(two_mo, 1, 3)

core_constant = np.array([rhf.energy_nuc()])

H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)

H = qml.jordan_wigner(H_fermionic)

(2, 27, 27)
(27, 27)
(27, 27, 27, 27)


ValueError: operand has more dimensions than subscripts given in einstein sum, but no '...' ellipsis provided to broadcast the extra dimensions.

In [29]:
len(rhf.mo_coeff)

2

In [3]:
print(rhf.mo_coeff.shape)  # Should be (num_AOs, num_MOs)
print(one_ao.shape)        # Check if > 2D


(2, 27, 27)
(27, 27)


## Trying Pennylane H building

In [37]:
from pyscf import gto, ao2mo, scf
import numpy as np
import pennylane as qml
mol_pyscf = gto.M(atom = '''H -0.02111417 -0.00201087  0.;
                            O  0.83504162  0.45191733  0.;
                            H  1.47688065 -0.27300252  0.''')
rhf = scf.RHF(mol_pyscf)
energy = rhf.kernel()

converged SCF energy = -74.9645256918536


In [38]:
one_ao = mol_pyscf.intor_symmetric('int1e_kin') + mol_pyscf.intor_symmetric('int1e_nuc')
two_ao = mol_pyscf.intor('int2e_sph') #2-electron integrals 

one_mo = np.einsum('pi,pq,qj->ij', rhf.mo_coeff, one_ao, rhf.mo_coeff)
two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

two_mo = np.swapaxes(two_mo, 1, 3)

core_constant = np.array([rhf.energy_nuc()])
H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)
H = qml.jordan_wigner(H_fermionic)


In [41]:
two_mo.shape

(7, 7, 7, 7)

## Li code from net

In [32]:
import numpy as np
from pyscf import gto, scf, ao2mo

# 1. Define the molecule with open-shell configuration (e.g., Li atom)
mol = gto.M(atom='Li 0 0 0', basis='sto-3g', spin=1) # Spin=1 indicates a doublet

# 2. Choose the UHF method
mf = scf.UHF(mol)

# 3. Perform the SCF calculation
mf.kernel()

# 4. Obtain the integrals in the AO basis
hcore_ao = mol.intor_symmetric('int1e_kin') + mol.intor_symmetric('int1e_nuc')
eri_ao = mol.intor('int2e') # Using 8-fold symmetry

# 5. Transform the integrals to the MO basis
mo_coeff = mf.mo_coeff # Molecular orbitals from UHF calculation

hcore_mo_alpha = np.einsum('pi,pq,qj->ij', mo_coeff[0], hcore_ao, mo_coeff[0])
hcore_mo_beta = np.einsum('pi,pq,qj->ij', mo_coeff[1], hcore_ao, mo_coeff[1])

eri_mo_alpha = ao2mo.incore.full(eri_ao, mo_coeff[0])
eri_mo_beta = ao2mo.incore.full(eri_ao, mo_coeff[1])
eri_mo_alpha_beta = ao2mo.incore.general(eri_ao, (mo_coeff[0], mo_coeff[0], mo_coeff[1], mo_coeff[1]))  #Changed Full to general

# The Hamiltonian can then be constructed using these MO basis integrals


converged SCF energy = -7.31552598128109  <S^2> = 0.75  2S+1 = 2


## Working code

In [51]:
import numpy as np
from pyscf import gto, scf, ao2mo

# 1. Define the molecule with open-shell configuration (e.g., Li atom)
mol = gto.M(atom='Fe 0 0 0; S 0 0 1.826', basis=basis_dict, spin=4) # Spin=1 indicates a doublet

# 2. Choose the UHF method
mf = scf.UHF(mol)

# 3. Perform the SCF calculation
mf.kernel()

# 4. Obtain the integrals in the AO basis
hcore_ao = mol.intor_symmetric('int1e_kin') + mol.intor_symmetric('int1e_nuc')
eri_ao = mol.intor('int2e') # Using 8-fold symmetry

# 5. Transform the integrals to the MO basis
mo_coeff = mf.mo_coeff # Molecular orbitals from UHF calculation

hcore_mo_alpha = np.einsum('pi,pq,qj->ij', mo_coeff[0], hcore_ao, mo_coeff[0])
hcore_mo_beta = np.einsum('pi,pq,qj->ij', mo_coeff[1], hcore_ao, mo_coeff[1])

#Generate hcore_mo
N = hcore_mo_alpha.shape[0]
print(N)
one_mo = np.zeros((2*N, 2*N))
print(one_mo.shape)
one_mo[:N, :N] = hcore_mo_alpha
one_mo[N:, N:] = hcore_mo_beta

eri_mo_alpha = ao2mo.incore.full(eri_ao, mo_coeff[0])
eri_mo_beta = ao2mo.incore.full(eri_ao, mo_coeff[1])
eri_mo_alpha_beta = ao2mo.incore.general(eri_ao, (mo_coeff[0], mo_coeff[0], mo_coeff[1], mo_coeff[1]))  #Changed Full to general

# The Hamiltonian can then be constructed using these MO basis integrals
two_mo = np.swapaxes(eri_mo_alpha_beta, 1,3)

core_constant = np.array([mf.energy_nuc()])
#H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)
#H = qml.jordan_wigner(H_fermionic)


converged SCF energy = -1655.10155370991  <S^2> = 6.0386361  2S+1 = 5.0154306
27
(54, 54)


In [ ]:
one_ao = mol_pyscf.intor_symmetric('int1e_kin') + mol_pyscf.intor_symmetric('int1e_nuc')
two_ao = mol_pyscf.intor('int2e_sph') #2-electron integrals 

one_mo = np.einsum('pi,pq,qj->ij', rhf.mo_coeff, one_ao, rhf.mo_coeff)
print(one_mo.shape)
two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)
print(two_mo.shape)

two_mo = np.swapaxes(two_mo, 1, 3)



core_constant = np.array([rhf.energy_nuc()])
H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)
H = qml.jordan_wigner(H_fermionic)